In [5]:
import pandas as pd
import boto3
import sagemaker


In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()


In [7]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

FileNotFoundError: [Errno 2] No such file or directory: 'plagiarism_data'

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

sagemaker/sentiment_rnn/train.csv
sagemaker/sentiment_rnn/word_dict.pkl
Test passed!


In [9]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

Error: cannot read infile: [Errno 2] No such file or directory: 'source_sklearn/train.py'


In [10]:
from sagemaker import LinearLearner

output_path = 's3://{}/{}'.format(bucket, prefix)

estimator = LinearLearner(role=role,
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    predictor_type="binary_classifier",
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    epochs=15)

In [11]:
# Load train and test data
import os
import numpy as np

train = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None)

train_y = train.loc[:,0]
train_x = train.loc[:, 1:3]

FileNotFoundError: [Errno 2] File b'plagiarism_data/train.csv' does not exist: b'plagiarism_data/train.csv'

In [12]:
# Convert to RecordSet
train_x_np = np.array(train_x).astype('float32')
train_y_np = np.array(train_y).astype('float32')

formatted_train_data = estimator.record_set(train_x_np, labels=train_y_np)

NameError: name 'train_x' is not defined

In [13]:
%%time

# Train your estimator on S3 training data
estimator.fit(formatted_train_data)

NameError: name 'formatted_train_data' is not defined

In [14]:
%%time

# deploy your model to create a predictor
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

ValueError: Estimator is not associated with a training job

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]



FileNotFoundError: [Errno 2] File b'plagiarism_data/test.csv' does not exist: b'plagiarism_data/test.csv'

In [16]:
# First: generate predicted, class labels
test_x = np.array(test_x).astype('float32')
test_y = np.array(test_y)
test_y_preds = predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

NameError: name 'test_x' is not defined

In [17]:
# Pull out predictions
test_y_preds = np.array([x.label['predicted_label'].float32_tensor.values[0] for x in test_y_preds])
test_y_preds

NameError: name 'test_y_preds' is not defined

In [18]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y)

NameError: name 'test_y' is not defined

In [19]:
# uncomment and fill in the line below!
estimator.delete_endpoint()


ValueError: Endpoint was not created yet

In [20]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'D4834E623AC68BBE',
   'HostId': 'Y7+HJRsHy9D7RmiIYAr8nWRbfw9uqjviTx/K7CkxInkwWfvRohHM28yrSKrduj2NtbePvAHQrv8=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'Y7+HJRsHy9D7RmiIYAr8nWRbfw9uqjviTx/K7CkxInkwWfvRohHM28yrSKrduj2NtbePvAHQrv8=',
    'x-amz-request-id': 'D4834E623AC68BBE',
    'date': 'Fri, 27 Sep 2019 07:31:08 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/sentiment_rnn/word_dict.pkl'},
   {'Key': 'sagemaker/sentiment_rnn/train.csv'}]}]